In [11]:
import pandas as pd
from GPTModel import GPTInference
import spacy
import re, string
import nltk
nlp = spacy.load("en_core_web_sm")

In [12]:
df = pd.read_csv('/home/sanjana/factual_evaluation_source_based/datasets/llm_annotations/sample/sample_annotations_sanjana.csv')
# df = df[df['origin'] != 'RefMatters_DialogSu']
df.head()

,Unnamed: 0,uuid,user_id,docid,model,nonfactual_spans,evidence,error_type,summary,dialogue
0,16,3cc5013c-4080-413e-aa5f-aec0dc93e349,sanjana,13681870,gpt4-32k-0613,NaN,NaN,NaN,Miley expresses her fatigue and lack of desire...,Miley: don''t want to go to work tomorrow!\r\n...
1,27,c094b128-08b6-4187-91cf-c44d9ab25baa,sanjana,13681871,gpt4-32k-0613,NaN,NaN,NaN,Andy informs Sue that he will be working late ...,Andy: Working late til 4 xx\r\nSue: no worries...
2,32,149452c6-bf79-47b0-b0ee-ecfcf99caa7f,sanjana,13681429,gpt4-32k-0613,NaN,NaN,NaN,Cass confirmed with Jordan that he received hi...,"Cass: Hi darling, did you get your birthday ca..."
3,34,a8c98d99-c964-46d0-a51c-bca6569900c1,sanjana,13681439,gpt4-32k-0613,NaN,NaN,NaN,Phoebe can''t go out because her mother is ang...,Phil: can you go out today?\r\nPhoebe: no\r\nP...
4,36,48d59334-9492-4cc1-a5ce-1b0378e34a1f,sanjana,13681441,gpt4-32k-0613,NaN,NaN,NaN,Joanna shares with Merve that their mutual acq...,Joanna: They are sending emails about Lewandow...


In [96]:

def get_atomic_facts_gpt(gpt_model_atomic, text, text_type):
    instr = f'Segment the following {text_type} into facts without introducing any unsupported information'
    prompt = f'{instr}\nDialogue: {text}'
    print(prompt)
    gpt_response = gpt_model_atomic.get_chatgpt_response(prompt)
    return gpt_response

def mask_all_errtypes(text):
    doc = nlp(text)
    origin_tokens = [token for token in doc]
    all_masked_results = []
    mask_idx = []
    mask_tokens = []
    for idx, token in enumerate(doc):
        append = False
        if 'subj' in token.dep_ or 'obj' in token.dep_:
            # print(idx, origin_tokens)
            # masked_text = [each.text for each in origin_tokens[:idx]] + ['<BLANK>'] + [each.text for each in origin_tokens[idx+1:]]
            # masked_text = ' '.join(masked_text)
            # print(masked_text)
            append = True
            token_type = 'subjobj'
            
        elif 'VERB' == token.pos_:
            append = True 
            token_type = 'predicate'

        elif 'ADV' == token.pos_:
            append = True
            token_type = 'circumstance'

        elif 'ADP' == token.pos_:
            append = True
            token_type = 'circumstance'

        elif 'PRON' == token.pos_:
            append = True
            token_type = 'coreference'

        

        if append:
            mask_idx.append(idx)
            mask_tokens.append((idx, token.text, token_type))
    # print([])
    # print(mask_tokens)  

    error_types_indices_map = {}
    for token_idx, token_text, err_type in mask_tokens:
        if err_type not in error_types_indices_map:
            error_types_indices_map[err_type] = []
        error_types_indices_map[err_type] += [token_idx]
        
    for err_type, err_indices in error_types_indices_map.items():
        for idx in err_indices:
            masked_text = [each.text for each in origin_tokens[:idx]] + ['<BLANK>'] + [each.text for each in origin_tokens[idx+1:]]
            masked_text = ' '.join(masked_text)
            all_masked_results.append((masked_text, origin_tokens[idx].text, err_type))
    return all_masked_results

def get_atomic_facts_gpt(gpt_model_atomic, model, text, text_type):
    instr = f'Write the {text_type} into indirect speech without introducing any unsupported information or inferences'
    prompt = f'{instr}\nDialogue: {text}'
    print(prompt)
    gpt_response = gpt_model_atomic.get_chatgpt_response(prompt, model = model)
    # print(nltk.sent_tokenize(gpt_response))
    return gpt_response


def get_atomic_facts_gpt(gpt_model_atomic, model, text, text_type):
    instr = f'Segment the following {text_type} into facts without introducing any unsupported information'
    prompt = f'{instr}\nDialogue: {text}'
    print(prompt)
    gpt_response = gpt_model_atomic.get_chatgpt_response(prompt)
    return gpt_response

def get_atomic_facts(gpt_model_atomic, dlg):
        atomic_facts = get_atomic_facts_gpt(gpt_model_atomic, 'gpt-4-32k-0613', dlg, 'dialogue')
        atomic_facts = nltk.sent_tokenize(atomic_facts)
        atomic_facts = [re.sub('[1-9]', '', each) for each in atomic_facts]
        atomic_facts = [each.strip(string.punctuation).strip() for each in atomic_facts]
        return atomic_facts

In [97]:
gpt_model_corruptor = GPTInference()
gpt_model_sentence_checker = GPTInference()
gpt_model_atomic = GPTInference()
gpt_model_nli = GPTInference()

In [334]:
row = df[~df['nonfactual_spans'].isnull()].sample(1)
text = row['summary'].values[0]
dlg = row['dialogue'].values[0]
print(row['dialogue'].values[0])
print(row['summary'].values[0])
print(row['nonfactual_spans'].values[0])

Julie: hey guys... could you just talk to me for a bit? I just watched this Japanese horror movie and I''m home alone and a little uneasy (aka scared shitless)
Rose: Jesus, why on earth would you watch a Japanese horror, home alone at this hour?
Julie: Cause I''m a fucking moron?
Rose: Cause you''re a fuckin moron. 
Paula: 5 point for Gryffindor, my friend. But in your defense, I always thought you were quite unaffected by horror movies.
Julie: That''s what I thought too!
Paula: So what kinda movie was it?
Julie: <file_other>
Rose: seems pretty generic
Julie: It''s scarier than it looks.
Rose: you know that japanese horror are the worst ones, right?
Julie: I know that now.
Paula: If... you want me to come over I can.
Julie: omg really? <3
Paula: Yeah, sure, it''s like 20 minutes by bike to your house, we can drink cocoa and watch "when harry met Sally" until we fall asleep ;)
Julie: Omg Thank you so much, I love you!
Rose: I wish I lived in your neighborhood :(
Paula: We can chip in fo

In [335]:
row

,Unnamed: 0,uuid,user_id,docid,model,nonfactual_spans,evidence,error_type,summary,dialogue
166,203,6d7363bb-1374-48da-9156-f06a609e8e58,sanjana,13821073,gpt4-32k-0613,a non-scary movie,Not found,Extrinsic_Error,After watching a Japanese horror movie late at...,Julie: hey guys... could you just talk to me f...


In [343]:
atomic_facts = get_atomic_facts(gpt_model_atomic, dlg)
atomic_facts = ' '.join(atomic_facts)

Segment the following dialogue into facts without introducing any unsupported information
Dialogue: Julie: hey guys... could you just talk to me for a bit? I just watched this Japanese horror movie and I''m home alone and a little uneasy (aka scared shitless)
Rose: Jesus, why on earth would you watch a Japanese horror, home alone at this hour?
Julie: Cause I''m a fucking moron?
Rose: Cause you''re a fuckin moron. 
Paula: 5 point for Gryffindor, my friend. But in your defense, I always thought you were quite unaffected by horror movies.
Julie: That''s what I thought too!
Paula: So what kinda movie was it?
Julie: <file_other>
Rose: seems pretty generic
Julie: It''s scarier than it looks.
Rose: you know that japanese horror are the worst ones, right?
Julie: I know that now.
Paula: If... you want me to come over I can.
Julie: omg really? <3
Paula: Yeah, sure, it''s like 20 minutes by bike to your house, we can drink cocoa and watch "when harry met Sally" until we fall asleep ;)
Julie: Omg 

In [344]:
atomic_facts

' Julie watched a Japanese horror movie while home alone  Julie feels scared after watching the movie  Rose asks Julie why she would watch a horror movie alone at night  Julie jokes about being a moron for watching the horror movie alone  Paula awards  points to Julie and mentions that she thought Julie was unaffected by horror movies  Julie also thought she was unaffected by horror movies  Paula asks about the type of movie Julie watched  Julie provides a file related to the movie she watched  Rose comments that the movie seems generic 0 Julie states that the movie was scarier than it looks  Rose says that Japanese horror movies are the worst  Julie admits that she now knows how scary Japanese horror movies can be  Paula offers to come over to Julie\'s house  Julie is thankful and accepts Paula\'s offer  Paula points out that it takes 0 minutes by bike to get to Julie\'s house and suggests they can watch "When Harry Met Sally" and drink cocoa  Julie expresses her appreciation and love

In [352]:
instr = 'Extract and list all subject/object words or phrases from the summary'
prompt = f'{instr}\nSummary:{text}\nList:'
answers = GPTInference().get_chatgpt_response(prompt)
answers

"1. Japanese horror movie\n2. Night\n3. Home\n4. Julie\n5. Friends\n6. Rose\n7. Paula\n8. Comfort\n9. Horror movie\n10. Intensity\n11. Julie's house\n12. Company\n13. Non-scary movie\n14. Cocoa\n15. Gratitude\n16. Spontaneous sleepover\n17. Late hour\n18. Cookies"

In [346]:
print(prompt)

Extract and list all subject/object words or phrases from the summary
Summary:After watching a Japanese horror movie late at night while home alone, Julie is left feeling scared and reaches out to her friends Rose and Paula for comfort. Rose criticizes Julie for watching a horror movie alone, despite being aware of their intensity. Paula offers to come over to Julie''s house to keep her company, suggesting they watch a non-scary movie and have some cocoa. Julie expresses gratitude and invites Rose over as well for a spontaneous sleepover. Despite the late hour, Rose agrees to come, promising to bring cookies.
List:


In [353]:
answers = [re.sub('[1-9]?[1-9].', '', each) for each in answers.split('\n')]
answers = [each.strip(string.punctuation) for each in answers]
answers = [each.strip() for each in answers if each in text or each.lower() in text.lower()]

answers

['Japanese horror movie',
 'Night',
 'Home',
 'Julie',
 'Friends',
 'Rose',
 'Paula',
 'Comfort',
 'Horror movie',
 'Intensity',
 'Company',
 'Non-scary movie',
 'Cocoa',
 'Gratitude',
 'Spontaneous sleepover',
 'Late hour',
 'Cookies']

In [363]:
instr = 'Extract and list all predicate words or phrases from the summary'
prompt = f'{instr}\nSummary:{text}\nList:'
answers = GPTInference().get_chatgpt_response(prompt)
answers

'- watching\n- is left feeling\n- reaches out \n- criticizes\n- being aware of\n- offers to come over\n- suggesting\n- watch\n- have\n- expresses gratitude \n- invites\n- agrees to come\n- promising to bring'

In [364]:
answers = [re.sub('[1-9]?[1-9].', '', each) for each in answers.split('\n')]
answers = [each.strip(string.punctuation) for each in answers]
answers = [each.strip() for each in answers if each in text or each.lower() in text.lower()]

answers

['watching',
 'is left feeling',
 'reaches out',
 'criticizes',
 'being aware of',
 'offers to come over',
 'suggesting',
 'watch',
 'have',
 'expresses gratitude',
 'invites',
 'agrees to come',
 'promising to bring']

In [365]:
masked_text = re.sub(answers[4], '<BLANK>', text)
if (masked_text == text):
    masked_text = re.sub(answers[4].lower(), '<BLANK>', text.lower())
masked_text

"After watching a Japanese horror movie late at night while home alone, Julie is left feeling scared and reaches out to her friends Rose and Paula for comfort. Rose criticizes Julie for watching a horror movie alone, despite <BLANK> their intensity. Paula offers to come over to Julie''s house to keep her company, suggesting they watch a non-scary movie and have some cocoa. Julie expresses gratitude and invites Rose over as well for a spontaneous sleepover. Despite the late hour, Rose agrees to come, promising to bring cookies."

In [366]:
prompt = f'Fill in the <BLANK> in the summary with information extracted from the source\nSource:{atomic_facts}\nSummary:{masked_text}\nAnswer:'
questions = GPTInference().get_chatgpt_response(prompt)
questions = questions.split('\n')
questions

["Julie's previous belief that she was unaffected by"]

In [362]:
print(prompt)

Fill in the <BLANK> in the summary with information extracted from the source
Source: Julie watched a Japanese horror movie while home alone  Julie feels scared after watching the movie  Rose asks Julie why she would watch a horror movie alone at night  Julie jokes about being a moron for watching the horror movie alone  Paula awards  points to Julie and mentions that she thought Julie was unaffected by horror movies  Julie also thought she was unaffected by horror movies  Paula asks about the type of movie Julie watched  Julie provides a file related to the movie she watched  Rose comments that the movie seems generic 0 Julie states that the movie was scarier than it looks  Rose says that Japanese horror movies are the worst  Julie admits that she now knows how scary Japanese horror movies can be  Paula offers to come over to Julie's house  Julie is thankful and accepts Paula's offer  Paula points out that it takes 0 minutes by bike to get to Julie's house and suggests they can watch 

In [282]:
prompt = f'Source: {atomic_facts}\nQuestion:{questions[5]}\nAnswer using the source in few words\nAnswer:'
qa = GPTInference().get_chatgpt_response(prompt)
qa = qa.split('\n')
qa

IndexError: list index out of range

In [237]:
prompt

'Source:  Lee is scared and confused  The digital wizard client manager, who normally only wears hoodies, is wearing a dress shirt and suit jacket  Lee makes a joke that the Russians are coming  Andy suggests the digital wizard client manager might have a date after work  Andy makes a joke that the date may be with a Russian soldier  Lee guesses that the manager may have a client-related presentation  Lee comments that the situation is unusual  Lee has been working at the office for almost five months  This is the first time Lee has seen the manager dressed in formal wear 0 Andy suggests that Lee take pictures of the manager and make fun of him later  Lee agrees and decides to take a quick picture and post it on their corporate Facebook page  Lee purchased RoS CE digitally in a spur-of-the-moment decision  Buying RoS CE unlocked some features for Lee such as a pet for WoW, decals for SC, and a hero for an unreleased game  Andy points out that those features would be available anyway fo

In [252]:
print(dlg)

Lee: I''m scared and confused. The digital wizard client manager who normally wears only hoodies, just put on a proper dress shirt and a suit jacket.
Lee: The Russians are coming!
Andy: Maybe he has a date after work?
Andy: Possibly with a Russian soldier ;)
Lee: If I''m not mistaken, he has some sort of a client related presentation.
Lee: But it''s still weird.
Lee: I''ve been here for almost 5 months and this is the first time I see something like this. :D
Andy: Take pictures and make fun of him in a couple of weeks :P
Lee: I should.
Lee: Quick picture and post to FB.
Andy: The corporate one!
Lee: Yeah!
Lee: But on a more interesting topic, I went and pre-purchased RoS CE in a temporal break of my brains.
Andy: Digital or the "real" one?
Lee: Digital.
Andy: Did it unlock any features already?
Lee: Pet for WoW, some decals for SC2.
Lee: Pre-ordering opened up a hero in a game not yet released. :D
Andy: So basically you got some goodies that you would get anyway by buying it closer to 

In [234]:
text

'Lee discusses his surprise at seeing a normally casual-dressed colleague wearing a dress shirt and suit jacket to work, jokingly suggesting that the strange occurrence could signal "The Russians are coming." Andy teasingly suggests the individual might have a date later, or a client presentation. Lee then mentions pre-purchasing a digital copy of RoS CE, which resulted in bonus features such as a pet for World of Warcraft, decals for Starcraft 2, and a hero in an unreleased game. Lee and Andy briefly argue about the value of pre-order bonuses with Andy suggesting that Lee would get the same bonuses if he bought the game closer to the release date.'

In [216]:
atomic_facts = ' '.join(atomic_facts)